In [1]:
import xarray as xr
import numpy as np
import rioxarray
from rasterio.enums import Resampling
import glob
import pandas as pd
import time
from dask.diagnostics import ProgressBar
import xesmf as xe
import dask.distributed

dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/"

in_dir = dir + ("data/processed_data/landsat/mosaic/")
out_dir = dir + ("data/processed_data/landsat/resampled/")

target_image = xr.open_dataset(
    dir +
    "data/processed_data/noaa_nc/lai_fapar/resampled/lai_annual_resample_max_1984_2013.nc"
)["LAI"].isel(time=5)
arr = xr.open_dataarray("../data/arr_id.nc")
target_image = target_image.rio.write_crs("EPSG:4326").rename({
    "latitude": "lat",
    "longitude": "lon"
})
# Grid of the target image 
lat1 = target_image["lat"].values
lon1 = target_image["lon"].values

Resmpling TM and ETM NDVI and make the values between the two sensors consistent

In [ ]:
for i in np.arange(1995,2015):
    print("resampling NDVI TM of the year : "+str(i))
    ref_image = xr.open_dataarray(in_dir + "mosaic_NDVI_"+str(i)+"_TM5.tif", engine="rasterio",chunks={"y":2500,"x":2500})
    ref_reproj = ref_image.rio.reproject(4326,esampling=Resampling.nearest)
    ref_reproj = ref_reproj.rename({"y": "lat","x": "lon"}).squeeze()
    var_regridded = ref_reproj.interp(lat=lat1,lon=lon1,method='linear')
    var_regridded.to_netcdf(dir+"data/processed_data/landsat/resampled/NDVI_resampled_"+str(i)+"_TM_v2.nc")

In [ ]:
for i in np.arange(1999,2015):
    print("Correcting and resampling ETM of the year : "+str(i))
    ref_image = xr.open_dataarray(in_dir + "mosaic_NDVI_"+str(i)+"_ETM.tif", engine="rasterio",chunks={"y":2500,"x":2500})
    ref_image = ref_image/10000
    ref_image_cor = (ref_image+0.015)/1.095
    ref_image_cor = ref_image_cor*10000
    ref_reproj = ref_image_cor.rio.reproject(4326,esampling=Resampling.nearest)
    ref_reproj = ref_reproj.rename({"y": "lat","x": "lon"}).squeeze()
    var_regridded = ref_reproj.interp(lat=lat1,lon=lon1,method='linear')
    var_regridded.to_netcdf(dir+"data/processed_data/landsat/resampled/NDVI_resampled_"+str(i)+"_ETMCor_v2.nc")

Correcting and resampling ETM of the year : 1999
Correcting and resampling ETM of the year : 2000
Correcting and resampling ETM of the year : 2001
Correcting and resampling ETM of the year : 2002
Correcting and resampling ETM of the year : 2003
Correcting and resampling ETM of the year : 2004
Correcting and resampling ETM of the year : 2005
Correcting and resampling ETM of the year : 2006
Correcting and resampling ETM of the year : 2007
Correcting and resampling ETM of the year : 2008
Correcting and resampling ETM of the year : 2009
Correcting and resampling ETM of the year : 2010
Correcting and resampling ETM of the year : 2011
Correcting and resampling ETM of the year : 2012
Correcting and resampling ETM of the year : 2013
Correcting and resampling ETM of the year : 2014


In [ ]:
# Correcting for large values outside of the region. 
for i in range(1999,2015):
    print(i)
    da=xr.open_dataset(dir+"data/processed_data/landsat/resampled/NDVI_resampled_"+str(i)+"_ETMCor_v2.nc")["band_data"]
    da = da.where(da<10000)
    da.to_netcdf(dir+"data/processed_data/landsat/resampled/NDVI_resampled_"+str(i)+"_ETMCor_v3.nc")

fnames= []
for i in np.arange(1984,2015):
    if i<=1999:
        fnames.append(dir+"data/processed_data/landsat/resampled/NDVI_resampled_"+str(i)+"_TM_v2.nc")
        continue
    fnames.append(dir+"data/processed_data/landsat/resampled/NDVI_resampled_"+str(i)+"_ETMCor_v3.nc") 
t=pd.date_range(start="1984",end="2015",freq="A").year
ds = xr.open_mfdataset(fnames,concat_dim="time",combine="nested").rename_vars({"band_data":"NDVI"})
ds["time"]=t
da = ds["NDVI"]
da.to_netcdf(dir+"data/processed_data/landsat/resampled/NDVI_resampled_v2.nc")

1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014


Resmpling TM and ETM NIR and make the values between the two sensors consistent

In [4]:
for i in np.arange(1995,2015):
    print("resampling NIR TM of the year : "+str(i))
    ref_image = xr.open_dataarray(in_dir + "mosaic_NIR_"+str(i)+"_TM5.tif", engine="rasterio",chunks={"y":2500,"x":2500})
    ref_reproj = ref_image.rio.reproject(4326,esampling=Resampling.nearest)
    ref_reproj = ref_reproj.rename({"y": "lat","x": "lon"}).squeeze()
    var_regridded = ref_reproj.interp(lat=lat1,lon=lon1,method='linear')
    var_regridded.to_netcdf(dir+"data/processed_data/landsat/resampled/NIR_resampled_"+str(i)+"_TM_v2.nc")

resampling NIR TM of the year : 1995
resampling NIR TM of the year : 1996
resampling NIR TM of the year : 1997
resampling NIR TM of the year : 1998
resampling NIR TM of the year : 1999
resampling NIR TM of the year : 2000
resampling NIR TM of the year : 2001
resampling NIR TM of the year : 2002
resampling NIR TM of the year : 2003
resampling NIR TM of the year : 2004
resampling NIR TM of the year : 2005
resampling NIR TM of the year : 2006
resampling NIR TM of the year : 2007
resampling NIR TM of the year : 2008
resampling NIR TM of the year : 2009


In [ ]:
for i in np.arange(1999,2015):
    print("Correcting and resampling NIR ETM of the year : "+str(i))
    ref_image = xr.open_dataarray(in_dir + "mosaic_NIR_"+str(i)+"_ETM.tif", engine="rasterio",chunks={"y":2500,"x":2500})
    ref_image = ref_image/10000
    ref_image_cor = (ref_image+0.004)/1.052
    ref_image_cor = ref_image_cor*10000
    ref_reproj = ref_image_cor.rio.reproject(4326,esampling=Resampling.nearest)
    ref_reproj = ref_reproj.rename({"y": "lat","x": "lon"}).squeeze()
    var_regridded = ref_reproj.interp(lat=lat1,lon=lon1,method='linear')
    var_regridded.to_netcdf(dir+"data/processed_data/landsat/resampled/NDVI_resampled_"+str(i)+"_ETMCor_v2.nc")